In [1]:
%load_ext lab_black
%cd ../../..

/data3/SIG/log-analytics


In [2]:
import torch
import numpy as np
import pandas as np
import pickle
from tqdm import tqdm
import sys

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
import time

time.sleep(3600 * 4)

In [5]:
deck1 = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3-deck2.pth")

In [6]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [7]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [8]:
N = deck2["feat"].size(0)
distdeck = {"dist": [], "tlevel": [], "fclevel": [], "index": [], "ids": []}
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][i, None], p=None, dim=1)
        dist_, indices_ = dist_.topk(4, largest=False)
        tlevels = deck1["tlevel"][indices_]
        fclevels = deck1["fclevel"][indices_]

        distdeck["dist"].append(dist_.cpu())
        distdeck["tlevel"].append(tlevels)
        distdeck["fclevel"].append(fclevels)
        distdeck["index"].append(indices_.cpu())
        distdeck["ids"].append(deck2["ids"][i])

        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [2:49:21<00:00, 107.85it/s]


In [10]:
distdeck["dist"] = torch.stack(distdeck["dist"])
distdeck["tlevel"] = torch.stack(distdeck["tlevel"])
distdeck["fclevel"] = torch.stack(distdeck["fclevel"])
distdeck["index"] = torch.stack(distdeck["index"])

In [ ]:
torch.save(
    distdeck, "results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_3-distdeck.pth"
)